In [ ]:
import numpy as np
import cv2

import matplotlib.pyplot as plt
import matplotlib

def ROUND(x):
	if isinstance(x, list):
		return [int(round(xx)) for xx in x]
	else:
		return int(round(x))


##############################################################################3
kaggle_dir = '/kaggle/input/physionet-ecg-image-digitization'
processed_dir = '/kaggle/input/hengck23-gridpoint-demo'

image_id ='4211091537'



# show the reference gridpoint annotation

image_file = f'{kaggle_dir}/train/{image_id}/{image_id}-0001.png' 
image = cv2.imread(image_file, cv2.IMREAD_COLOR_RGB)
gridpoint = np.load(f'{processed_dir}/{image_id}-0001.gridpoint_xy.npy' )

def draw_annotation(image, gridpoint):
    
    H,W = image.shape[:2]
    overlay = image // 2
    overlay1 = image // 2
    
    for j in range(44):
        for i in range(57):
            x,y = gridpoint[j,i]
            if (x==0) &(y==0): continue
            
            y = ROUND(y)
            x = ROUND(x)
            
            cv2.circle(overlay, (x, y), radius=4, color=[0, 255, 0], thickness=-1)
            cv2.circle(overlay1, (x, y), radius=10, color=[0, 255, 0], thickness=-1)
    
    flat = gridpoint.reshape(-1,2)
    miny = ROUND(flat[:,1].min())
    maxy = ROUND(flat[:,1].max())
    minx = ROUND(flat[:,0].min())
    maxx = ROUND(flat[:,0].max())
    
    plt.imshow(np.hstack([image,overlay1]))
    plt.show()
    plt.imshow(overlay[maxy-500:maxy, minx: minx+1500])
    plt.show()
            
# show results
draw_annotation(image, gridpoint)

In [ ]:

image_file = f'{kaggle_dir}/train/{image_id}/{image_id}-0006.png' 
image = cv2.imread(image_file, cv2.IMREAD_COLOR_RGB)
gridpoint = np.load(f'{processed_dir}/{image_id}-0006.gridpoint_xy.npy' )
draw_annotation(image, gridpoint)


In [ ]:
image_file = f'{kaggle_dir}/train/{image_id}/{image_id}-0005.png' 
image = cv2.imread(image_file, cv2.IMREAD_COLOR_RGB)
gridpoint = np.load(f'{processed_dir}/{image_id}-0005.gridpoint_xy.npy' )
draw_annotation(image, gridpoint)

## Example usage in recification

In [ ]:
import numpy as np
import cv2

import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib


def rectify_image(
	image, gridpoint_xy
):
	H, W = 1700, 2200 #reference size
	H1,W1 = image.shape[:2]
	sparse_map = gridpoint_xy/[[[W1-1,H1-1]]]*2 -1
	sparse_map = torch.from_numpy(np.ascontiguousarray(sparse_map.transpose(2, 0, 1))).unsqueeze(0).float()
	dense_map  = F.interpolate(sparse_map, size=(H, W), mode='bilinear', align_corners=True)
	distort    = torch.from_numpy(np.ascontiguousarray(image.transpose(2, 0, 1))).unsqueeze(0).float()
	rectified = F.grid_sample(
		distort, dense_map.permute(0, 2, 3, 1), mode='bilinear', padding_mode='border', align_corners=False
	)
	rectified = rectified.data.cpu().numpy()
	rectified = rectified[0].transpose(1, 2, 0).astype(np.uint8)
	return rectified

kaggle_dir = '/kaggle/input/physionet-ecg-image-digitization'
processed_dir = '/kaggle/input/hengck23-gridpoint-demo'

all=[]
image_id ='4211091537'
for type_id in ['0005', '0006', '0012']: 
    print('type_id:', type_id)

    image_file = f'{kaggle_dir}/train/{image_id}/{image_id}-{type_id}.png' 
    image = cv2.imread(image_file, cv2.IMREAD_COLOR_RGB)
    gridpoint_xy = np.load(f'{processed_dir}/{image_id}-{type_id}.gridpoint_xy.npy' ) #detected grid point from unet ,,,
    
    rectified = rectify_image(
    	image, gridpoint_xy
    )
    
    plt.imshow(image)
    plt.show()
    plt.imshow(rectified)
    plt.show()
    all.append(rectified[-1000:, 0:1000])

print('compare all results:')
plt.imshow(np.hstack(all), aspect='equal')
plt.show()